# PFC - ETL and Model Training Notebook - Rafael S. de Almeida

## Download python packages and import libraries

In [1]:
# %matplotlib inline
# %pip install psycopg2-binary

In [2]:
import numpy as np
import requests
import psycopg2
import pandas
import json
from scipy import spatial

## Call lightning forecast API

In [3]:
response = requests.get("https://ymh9rihek3.execute-api.sa-east-1.amazonaws.com/prod/LightningFunction")
lightningOccurrences = pandas.read_json (response.text)
# print (lightningOccurrences[['latitude','longitude']])

## Connect to DB and select towers info

In [4]:
PORT=              ""
DATABASE_USER=     ""
DATABASE_PASSWORD= ""
DATABASE_HOST=     ""
DATABASE_NAME=     ""
DATABASE_PORT=     ""

In [5]:
con = psycopg2.connect(host=DATABASE_HOST, database=DATABASE_NAME,user=DATABASE_USER, password=DATABASE_PASSWORD,
                       port=DATABASE_PORT)
cur = con.cursor()
sql = 'SELECT "ds_linha_transmissao", "coord_y", "coord_x" FROM "torres_completas" WHERE "coord_y" IS NOT NULL OR "coord_x" IS NOT NULL'
cur.execute(sql)
results = cur.fetchall()
towers = pandas.DataFrame (results,columns=['linha','latitude','longitude'])

## Define closest tower to each lightning and calculate the distance

In [6]:
tree = spatial.KDTree(towers[['latitude','longitude']])
distances = tree.query(lightningOccurrences[['latitude','longitude']])
distances = pandas.DataFrame(list(distances)).T
distances.columns = ['distance','towerIndex']

## Verify electrical overcurrent

In [7]:
results = []
for index, row in distances.iterrows():
    if row['distance'] <= 0.0005 and abs(int(lightningOccurrences[['corrente']].iloc[index])) >= 32.9:
        results.append(1)
    else:
        results.append(0)
results = pandas.DataFrame(list(results))

## Create training dataset

In [8]:
dataset = pandas.DataFrame({
    'distance': distances['distance'],
    'current' : lightningOccurrences['corrente'],
    'results' : results[0]
})

dataset

,distance,current,results
0,0.000382,45,1
1,0.000134,5,0
2,0.000342,45,1
3,0.000466,5,0
4,0.000127,45,1
...,...,...,...
322,0.097551,10,0
323,0.891642,4,0
324,0.273662,-5,0
325,1.862817,6,0
